In [1]:
from poisson_fem.mesh import PoissonFEM
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc

In [2]:
m = 8
mesh = PoissonFEM.RectangularMesh(np.ones(m)/m)
# mesh.plot()

In [3]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
mesh.setEssentialBoundary(origin)

In [4]:
def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

In [5]:
#Define boundary flux field
def flux(x):
    a = np.array([1, 2, 3])
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [6]:
rs = PoissonFEM.RightHandSide(mesh)

In [7]:
rs.compFluxBC(mesh, flux)

In [8]:
rs.fluxBC

array([[-0.203125 , -0.15625  , -0.1796875, -0.203125 , -0.2265625,
        -0.25     , -0.2734375, -0.296875 , -0.09375  ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       , -0.1171875,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       , -0.140625 ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       , -0.1640625,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        -0.1875   ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       , -0.2109375,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       , -0.234375 ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ],
       [-0.140625 , -0.1640625, -0.1875   , -0.2109375, -0.234375 ,
        -0.2578125, -0.28125  , -0.234375 ,  0.       ,  0.

In [9]:
funSpace = PoissonFEM.FunctionSpace(mesh)

In [10]:
K = PoissonFEM.StiffnessMatrix(mesh, funSpace)

In [11]:
# Precomputations
# Kvec = PETSc.Vec().createSeq(mesh.nEq**2)
x = PETSc.Vec().createSeq(mesh.nEq)
rhs = PETSc.Vec().createSeq(mesh.nEq)

In [12]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

In [13]:
# for scipy comparison
Kes = K.globStiffStencil.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

In [14]:
N = 1e2

In [15]:
# r1 = range(mesh.nCells)
r2 = range(mesh.nEq)
lmbda_np = np.ones(mesh.nCells)
rhs_np = np.ones(mesh.nEq)
start = time.time()
for n in range(int(N)):
    # Stiffness matrix assembly, lmbda = [1, 1, ..., 1]
    rhs.setValues(r2, rhs_np)
    K.assemble(lmbda_np)

    # solving
    ksp.setOperators(K.matrix)
    ksp.solve(rhs, x)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

PETSc time =  7.018804550170898e-05


In [16]:
start = time.time()
for n in range(int(N)):
    Kvecs = Kes @ np.ones(mesh.nCells)
    K1 = sps.csr_matrix((Kvecs[K.vec_nonzero], K.indices, K.indptr))
    x1 = lg.spsolve(K1, np.ones(mesh.nEq))
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

scipy time =  0.00033311843872070314


In [17]:
diff = np.linalg.norm(x.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)

result difference =  1.134057704558057e-14
PETSc speedup =  4.746085125174089


In [18]:
mesh.cells[0].containsEssentialVertex

True